# Préparation

## Imports librairies et data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
from os import listdir


from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, auc, roc_auc_score, roc_curve
from glob import glob



import time
import cv2
from PIL import Image



In [3]:
liste_file = []

try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        files = os.listdir("/content/drive/MyDrive/Notebooks/P6/Dataset/Images")
        path ="/content/drive/MyDrive/Notebooks/P6/Dataset/Images/"
        data_T =  pd.read_csv("/content/drive/MyDrive/Notebooks/P6/basep6.csv")

except ModuleNotFoundError : 
        files = os.listdir(r'C:\Users\Hugues\gdrive\Notebooks\P6\Dataset\Images')
        path ="C:\\Users\\Hugues\\gdrive\\Notebooks\\P6\\Dataset\\Images\\"
        data_T =  pd.read_csv("C:\\Users\\Hugues\\gdrive\\Notebooks\\P6\\basep6.csv")

for file in files:
    try:
        from google.colab import drive
        imgfile = "/content/drive/MyDrive/Notebooks/P6/Dataset/Images/"+file
        liste_file.append ( file)
        
    except ModuleNotFoundError :
        imgfile  = "C:\\Users\\Hugues\\gdrive\\Notebooks\\P6\\Dataset\\Images\\"+file
        liste_file.append ( file)




print (len(liste_file))


Mounted at /content/drive
1053


In [4]:
print (path) 

/content/drive/MyDrive/Notebooks/P6/Dataset/Images/


In [5]:
data_T.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 41 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Unnamed: 0.1                       1050 non-null   int64 
 1   Unnamed: 0                         1050 non-null   int64 
 2   uniq_id                            1050 non-null   object
 3   crawl_timestamp                    1050 non-null   object
 4   product_url                        1050 non-null   object
 5   product_name                       1050 non-null   object
 6   product_category_tree              1050 non-null   object
 7   pid                                1050 non-null   object
 8   retail_price                       1050 non-null   object
 9   discounted_price                   1050 non-null   object
 10  image                              1050 non-null   object
 11  is_FK_Advantage_product            1050 non-null   bool  
 12  descri

## Création des labels des photos à partir des category

In [6]:
data=data_T.sample(600) #sinon plante

#data=data_T
data=data.reset_index(drop = True)
data['image'] = data['image'].apply(lambda x: path+x)

In [7]:
data['image'].describe(include="all")

count                                                   600
unique                                                  600
top       /content/drive/MyDrive/Notebooks/P6/Dataset/Im...
freq                                                      1
Name: image, dtype: object

In [8]:
list_labels= data['category'].unique().tolist()

In [9]:
data["label"] = [list_labels.index(l) for l in data["category"]]

In [10]:
data[["label","category"]].sample(n = 20).sort_values('label')

,label,category
17,0,BeautyandPersonalCare
26,0,BeautyandPersonalCare
4,0,BeautyandPersonalCare
449,1,HomeDecor&FestiveNeeds
49,1,HomeDecor&FestiveNeeds
214,2,Watches
489,2,Watches
384,3,BabyCare
249,3,BabyCare
588,4,Kitchen&Dining


# création des descripteurs SIFT  de chaque image

In [11]:
import gc
gc.collect()

0

In [ ]:
# identification of key points and associated descriptors
import time, cv2
sift_keypoints = []
temps1=time.time()
#sift = cv2.xfeatures2d.SIFT_create(500)
sift = cv2.SIFT_create()

for image_num in range(data.shape[0]) : 
    if image_num%50 == 0 : 
        print(image_num)
        gc.collect()
    image1 = cv2.imread(data.loc[image_num, 'image'] )
    image = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)# convert in gray
    res = cv2.equalizeHist(image)   # equalize image histogram
    kp, des = sift.detectAndCompute(res, None)
    sift_keypoints.append(des)

sift_keypoints_by_img = np.asarray(sift_keypoints)
sift_keypoints_all    = np.concatenate(sift_keypoints_by_img, axis=0)

print()
print("Nombre de descripteurs : ", sift_keypoints_all.shape)

duration1=time.time()-temps1
print("temps de traitement SIFT descriptor : ", "%15.2f" % duration1, "secondes")

0
50
100
150
200
250
300
350


## Création des clusters de descripteurs
* Utilisation de MiniBatchKMeans pour obtenir des temps de traitement raisonnables

In [ ]:
from sklearn import cluster, metrics

# Determination number of clusters
temps1=time.time()

k = int(round(np.sqrt(len(sift_keypoints_all)),0))
print("Nombre de clusters estimés : ", k)
print("Création de",k, "clusters de descripteurs ...")

# Clustering
kmeans = cluster.MiniBatchKMeans(n_clusters=k, init_size=3*k, random_state=0)
kmeans.fit(sift_keypoints_all)

duration1=time.time()-temps1
print("temps de traitement kmeans : ", "%15.2f" % duration1, "secondes")

## Création des features des images
* Pour chaque image : 
   - prédiction des numéros de cluster de chaque descripteur
   - création d'un histogramme = comptage pour chaque numéro de cluster du nombre de descripteurs de l'image

In [ ]:
# Creation of histograms (features)
temps1=time.time()

def build_histogram(kmeans, des, image_num):
    res = kmeans.predict(des)
    hist = np.zeros(len(kmeans.cluster_centers_))
    nb_des=len(des)
    if nb_des==0 : print("problème histogramme image  : ", image_num)
    for i in res:
        hist[i] += 1.0/nb_des
    return hist


# Creation of a matrix of histograms
hist_vectors=[]

for i, image_desc in enumerate(sift_keypoints_by_img) : 
    if i%100 == 0 : print(i)  
    hist = build_histogram(kmeans, image_desc, i) #calculates the histogram
    hist_vectors.append(hist) #histogram is the feature vector

im_features = np.asarray(hist_vectors)

duration1=time.time()-temps1
print("temps de création histogrammes : ", "%15.2f" % duration1, "secondes")


Features d'une image = Histogramme d'une image = Comptage pour une image du nombre de descripteurs par cluster

In [ ]:
print ( len( sift_keypoints_by_img))

## Réductions de dimension

### Réduction de dimension PCA
* La réduction PCA permet de créer des features décorrélées entre elles, et de diminuer leur dimension, tout en gardant un niveau de variance expliquée élevé (99%)
* L'impact est une meilleure séparation des données via le T-SNE et une réduction du temps de traitement du T-SNE

In [ ]:
from sklearn import manifold, decomposition

print("Dimensions dataset avant réduction PCA : ", im_features.shape)
pca = decomposition.PCA(n_components=0.99)
feat_pca= pca.fit_transform(im_features)
print("Dimensions dataset après réduction PCA : ", feat_pca.shape)

### Réduction de dimension T-SNE
* Réduction de dimension en 2 composantes T-SNE pour affichage en 2D des images

In [ ]:
from sklearn import manifold, decomposition

tsne = manifold.TSNE(n_components=2, perplexity=30, 
                     n_iter=2000, init='random', random_state=6)
X_tsne = tsne.fit_transform(feat_pca)

df_tsne = pd.DataFrame(X_tsne[:,0:2], columns=['tsne1', 'tsne2'])
df_tsne["class"] = data["category"]
print(df_tsne.shape)


## Analyse visuelle : affichage T-SNE selon catégories d'images

* Les catégories  xxx sont partiellement séparées

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(
    x="tsne1", y="tsne2", hue="class", data=df_tsne, legend="brief",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.6)

plt.title('TSNE selon les vraies classes', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
#plt.legend(prop={'size': 14}) 
plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))

plt.show()


## Analyse mesures : similarité entre catégories et clusters

###  Création de clusters à partir du T-SNE

In [ ]:
from sklearn import cluster, metrics

cls = cluster.KMeans(n_clusters=7, random_state=22,n_init=100)
cls.fit(X_tsne)

df_tsne["cluster"] = cls.labels_
print(df_tsne.shape)


###  Affichage des images selon clusters et calcul ARI de similarité catégories images / clusters
* Le score ARI est

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(
    x="tsne1", y="tsne2",
    hue="cluster",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.6,
    data=df_tsne,
    legend="brief")

plt.title('TSNE selon les clusters', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
#plt.legend(prop={'size': 14}) 
plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
plt.show()

labels = data["label"]
print("ARI : ", metrics.adjusted_rand_score(labels, cls.labels_))

### Analyse par classes
* La matrice de confusion doit être transformée pour mettre avoir en colonne le même ordre des catégories supposées qu'en ligne
* Cette transformation est réalisé avec la fonction "argmax"

In [ ]:
df_tsne.groupby("cluster").count()["class"]

In [ ]:
conf_mat = metrics.confusion_matrix(labels, cls.labels_)
print(conf_mat)

In [ ]:
def conf_mat_transform(y_true,y_pred) :
    conf_mat = metrics.confusion_matrix(y_true,y_pred)
    
    #corresp = np.argmax(conf_mat, axis=0)
    corresp = [3,4,0,2,1,6,5]
    print ("Correspondance des clusters : ", corresp)
    #y_pred_transform = np.apply_along_axis(correspond_fct, 1, y_pred)
    labels = pd.Series(y_true, name="y_true").to_frame()
    labels['y_pred'] = y_pred
    labels['y_pred_transform'] = labels['y_pred'].apply(lambda x : corresp[x]) 
    
    return labels['y_pred_transform']

cls_labels_transform = conf_mat_transform(labels, cls.labels_)
conf_mat = metrics.confusion_matrix(labels, cls_labels_transform)
print(conf_mat)
print()
print(metrics.classification_report(labels, cls_labels_transform))

In [ ]:
df_cm = pd.DataFrame(conf_mat, index = [label for label in list_labels],
                  columns = [i for i in "0123456"])
plt.figure(figsize = (6,4))
sns.heatmap(df_cm, annot=True, cmap="Blues")

In [ ]:
## affichage exemples images et histogramme vectors

 ## affichage exemples images et histogramme vectors

In [ ]:
nb=54
image1 = cv2.imread(data.loc[nb, 'image'] )
image1=cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
plt.imshow(image1)
plt.show()

In [ ]:

# Affichage de l'histogramme 
plt.bar(range(len(hist_vectors[nb])), hist_vectors[nb])
plt.show()

In [ ]:
nb=42
image1 = cv2.imread(data.loc[nb, 'image'] )
image1=cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
plt.imshow(image1)
plt.show()

In [ ]:
# Affichage de l'histogramme 
plt.bar(range(len(hist_vectors[nb])), hist_vectors[nb])
plt.show()

In [ ]:
nb=22
image1 = cv2.imread(data.loc[nb, 'image'] )
image1=cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
plt.imshow(image1)
plt.show()

In [ ]:
# Affichage de l'histogramme 
plt.bar(range(len(hist_vectors[nb])), hist_vectors[nb])
plt.show()